# Implementation details and runtime comparisons

- Author: Marios Kokkodis
- email: marios.kokkodis@gmail.com 

> Python notes: tested on PY38


In [1]:
import sys

import pandas as pd
import numpy as np
if '../python/' not in sys.path: sys.path.insert(1, '../python/')
import time
from hmm_gbu import HMM
from custom_util_functions import do_feature_selection, run_logistic, get_auc_per_class, print_border_line
from custom_util_functions import  prepare_hmm_matrix, run_forest, run_xg, run_svm, get_model_key
from sklearn.pipeline import Pipeline
from hmm_functions import transform_input_to_HMM_timelines,createPriors,init_params
from sklearn import linear_model, svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore', 'Solver terminated early.*')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from joblib import dump, load



emission_variables = [ 'curRating','numberOfReviewsRest',
                'cheap','affordable','expensive','qualityOfRestaurantsVisited','focalUserReviews']
transition_variables = ['focalUserReviews']
transition_index = transition_variables.index('focalUserReviews')
fold = 0
dataset = 'restaurant_ncv.csv'
employer_id = 'user'
task_id = 'choicesetId'
application_id = 'application'
d = pd.read_csv("../../data/"+dataset)
d = d.sort_values([employer_id, 'daysTrend', task_id])
pipe = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy="median",add_indicator=False)),
                 ('scaler',MinMaxScaler())])
train = d[d['set_annotation'].str.contains(pat =('train_'+str(fold))+',|train_'+str(fold)+"$", regex=True)==True].copy()
validation = d[d['set_annotation'].str.contains(pat =('validation_'+str(fold))+',|validation_'+str(fold)+"$",
                                                regex=True)==True].copy()
trainedPipeline = pipe.fit(train[emission_variables])
trainTransformed = pd.DataFrame(data=trainedPipeline.transform(train[emission_variables]),
             index = None,  columns=emission_variables)
validation_transformed = pd.DataFrame(data=trainedPipeline.transform(validation[emission_variables]),
             index = None,  columns=emission_variables)
trainTransformed['set_annotation'] = 'train'
validation_transformed['set_annotation'] = 'validation'
train_y = train['Y_it']
validation_y = validation['Y_it']
transition_application_employer_train = prepare_hmm_matrix(trainTransformed[transition_variables],
                                                               train[application_id],
                                                               train[employer_id])
transition_application_employer_validation = prepare_hmm_matrix(validation_transformed[transition_variables],
                                                              validation[application_id],
                                                              validation[employer_id])

## Different solvers

#### COBYLA - 100 iterations

In [2]:
start_time = time.time()
model = HMM(transition_application_employer_train, transition_application_employer_validation, transition_index,
                     iterations=100,  states=4,
            verbal=True, solver = 'COBYLA')
Z_timelines = trainTransformed[emission_variables].to_numpy()
r = model.fit(Z_timelines, train_y.to_numpy())
print("\t> minimized ll:", round(r.optimization_result.fun))
print("--- %.3f seconds ---" % (time.time() - start_time))

#### COBYLA - 1000 iterations

-----------------------------------------------------
Note: The HMM assumes ranked timelines as inputs
-----------------------------------------------------
fit() was called...
Transforming input matrix to HMM sequences...
Total number of parameters to be estimated: 92
	> minimized ll: 803
--- 5.744 seconds ---


In [3]:
start_time = time.time()
model = HMM(transition_application_employer_train, transition_application_employer_validation, transition_index,
                     iterations=1000,  states=4,
            verbal=True, solver = 'COBYLA')
Z_timelines = trainTransformed[emission_variables].to_numpy()
r = model.fit(Z_timelines, train_y.to_numpy())
print("\t> minimized ll:", round(r.optimization_result.fun))
print("--- %.3f seconds ---" % (time.time() - start_time))

-----------------------------------------------------
Note: The HMM assumes ranked timelines as inputs
-----------------------------------------------------
fit() was called...
Transforming input matrix to HMM sequences...
Total number of parameters to be estimated: 92
	> minimized ll: 446
--- 56.889 seconds ---


#### BFGS - 100 iterations

In [4]:
start_time = time.time()
model = HMM(transition_application_employer_train, transition_application_employer_validation, transition_index,
                     iterations=100,  states=4,
            verbal=True, solver = 'BFGS')
Z_timelines = trainTransformed[emission_variables].to_numpy()
r = model.fit(Z_timelines, train_y.to_numpy())
print("\t> minimized ll:", round(r.optimization_result.fun))
print("--- %.3f seconds ---" % (time.time() - start_time))

-----------------------------------------------------
Note: The HMM assumes ranked timelines as inputs
-----------------------------------------------------
fit() was called...
Transforming input matrix to HMM sequences...
Total number of parameters to be estimated: 92
	> minimized ll: 1895
--- 297.966 seconds ---


#### L-BFGS - 100 iterations


In [5]:

start_time = time.time()
model = HMM(transition_application_employer_train, transition_application_employer_validation, transition_index,
                     iterations=100,  states=4,
            verbal=True, solver = 'L-BFGS-B')
Z_timelines = trainTransformed[emission_variables].to_numpy()
r = model.fit(Z_timelines, train_y.to_numpy())
print("\t> minimized ll:", round(r.optimization_result.fun))
print("--- %.3f seconds ---" % (time.time() - start_time))




-----------------------------------------------------
Note: The HMM assumes ranked timelines as inputs
-----------------------------------------------------
fit() was called...
Transforming input matrix to HMM sequences...
Total number of parameters to be estimated: 92
	> minimized ll: 826
--- 86.335 seconds ---


## Convergence analysis


Convergence test for COBYLA, BFGS, and LBFGS

In the following we use a tolerance level of 0.01, and allow for 100 iterations


In [6]:
for solver in ['COBYLA','BFGS','L-BFGS-B']:
    print_border_line()
    print("Using solver:",solver)
    start_time = time.time()
    model = HMM(transition_application_employer_train, transition_application_employer_validation, transition_index,
                     iterations=100,  states=2, tol = 0.01,
            verbal=True, solver = solver
        )
    Z_timelines = trainTransformed[emission_variables].to_numpy()
    r = model.fit(Z_timelines, train_y.to_numpy())
    print("\t> minimized ll:", round(r.optimization_result.fun))
    print("\t> convergence:",r.optimization_result.success)
    print("--- %.3f seconds ---" % (time.time() - start_time))


-----------------------------------------------------
Using solver: COBYLA
-----------------------------------------------------
Note: The HMM assumes ranked timelines as inputs
-----------------------------------------------------
fit() was called...
Transforming input matrix to HMM sequences...
Total number of parameters to be estimated: 38
	> minimized ll: 476
	> convergence: False
--- 5.553 seconds ---
-----------------------------------------------------
Using solver: BFGS
-----------------------------------------------------
Note: The HMM assumes ranked timelines as inputs
-----------------------------------------------------
fit() was called...
Transforming input matrix to HMM sequences...
Total number of parameters to be estimated: 38
	> minimized ll: 1496
	> convergence: True
--- 12.423 seconds ---
-----------------------------------------------------
Using solver: L-BFGS-B
-----------------------------------------------------
Note: The HMM assumes ranked timelines as inputs
-

## Complexity of likelihood estimation

#### Intractable raw likelihood

In [7]:
verbal=False
np.random.seed(1234)
states = [0,1]
#choice sets that include all outcomes
choice_sets = [73,   87,      78]
print("Total instances:", len(train[train.choicesetId.isin(choice_sets)]))
trainTransformed = pd.DataFrame(data=trainedPipeline.transform(train[train.choicesetId.isin(choice_sets)][emission_variables]),
             index = None,  columns=emission_variables)
train_y = train[train.choicesetId.isin(choice_sets)]['Y_it']
transition_application_employer_train = prepare_hmm_matrix(trainTransformed[transition_variables],
                                                    train[train.choicesetId.isin(choice_sets)][application_id],
                                                    train[train.choicesetId.isin(choice_sets)][employer_id])
Z_timelines = trainTransformed[emission_variables].to_numpy()
transition_model='multinomial_constrained'
X,y = Z_timelines, train_y.to_numpy()
emission_distro_pars = len(np.unique(y)) - 1
(sequences_X, sequences_Z, sequences_O, applications) = transform_input_to_HMM_timelines(X, y,
                        transition_application_employer_train,
                                verbal,n_cores=1)
variables_in_Z = sequences_Z[0].shape[1]
variables_in_X = sequences_X[0].shape[1]
# initialize parameters
(theta_prior, transitionParameters, emissionParameters, pi) = createPriors(variables_in_X,
                                                                                        variables_in_Z,
                                                                                        states,
                                                                                        transition_model,
                                                                                        emission_distro_pars,
                                                                                        verbal)

(pi, gammaCoeffsForZ, betaCoeffsforX,
     sigmaPars) = init_params(theta_prior, variables_in_Z, states,
                                                emission_distro_pars, variables_in_X,
                                                transition_model)

zCoeffsMat = []
for state in states:
    zCoeffsMat.append([])
    for symbol in range(emission_distro_pars + 1):  # number of symbols for multinomial emissions.
        zCoeffsMat[state].append(gammaCoeffsForZ[state][symbol])
zCoeffsMat = np.array(zCoeffsMat)

xCoeffsMat = []
for stateFrom in states:
    xCoeffsMat.append([])
    for stateTO in states:
        xCoeffsMat[stateFrom].append(betaCoeffsforX[stateFrom][stateTO])
xCoeffsMat = np.array(xCoeffsMat)

from importlib import reload
import hmm_functions
reload(hmm_functions)
from hmm_functions import transform_input_to_HMM_timelines, createPriors, init_params, get_individual_log_l


Total instances: 31


In [8]:
start_time = time.time()
#Returns the likelihood function by explicitly estimating the sums/products
L = 0
#sequences_X, sequences_Z, sequences_O
for i in range(len(sequences_X)):
    client_X = sequences_X[i]
    client_O = sequences_O[i]
    client_Z = sequences_Z[i]
    L-= get_individual_log_l(client_X,client_Z,client_O,
                            zCoeffsMat,None,transition_index,transition_model,
                            states,xCoeffsMat,pi)
print("Negative L naive: %.3f" %(L))
print("--- %.2f seconds ---" % (time.time() - start_time))

# %%

7.9416770404861435e-19
2.988449887486522e-08
Negative L naive: 59.003
--- 1037.36 seconds ---


## Vectorization

In [9]:
reload(hmm_functions)
from hmm_functions import get_likelihood_validation
start_time = time.time()
L= get_likelihood_validation(sequences_X,sequences_Z,sequences_O,pi,xCoeffsMat,zCoeffsMat,
                          transition_model,states,transition_index)
print("Negative L numpy: %.2f" %(L))
print("--- %.3f seconds ---" % (time.time() - start_time))


Negative L numpy: 59.00
--- 0.001 seconds ---



## Comparison with LSTM

In [10]:
!python ../python/test_speed.py

-----------------------------------------------------
Note: The HMM assumes ranked timelines as inputs
-----------------------------------------------------
fit() was called...
Transforming input matrix to HMM sequences...
Total number of parameters to be estimated: 63
Minimized ll: 2783
predict_proba() was called...
Transforming input matrix to HMM sequences...
-----------------------------------------------------
	Final AUC score on Hire-positive: 0.56605
-----------------------------------------------------
--- 98.163 seconds ---


In [11]:
!python ../python/test_speed_lstm.py -C




-----------------------------------------------------
2021-03-05 20:09:47.294379: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-05 20:09:47.294547: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-03-05 20:09:47.446208: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
-----------------------------------------------------
	(Final AUC score on Hire-positive: 0.531 )
-----------------------------------------------------
--- 125.947 seconds ---
